In [1]:
# getting data

In [1]:
import os
import re
import sys
import csv
import pysam
import shutil
import random
import requests
import subprocess
from pathlib import Path

from Bio import SeqIO, Align
from Bio.Align import substitution_matrices

import time

In [2]:
soft_dir = Path().resolve().parent / 'soft/vira/vira'
sys.path.append(str(soft_dir))

from utils.common import *

In [ ]:
base_dir = Path.cwd().parent.parent / 'HIV_Atlas_Data'
data_dir = base_dir / 'data'
outdir = data_dir / 'sequences'
outdir.mkdir(exist_ok=True)

reference_fasta_fname = data_dir / "reference/K03455.1.fasta"
reference_gtf_fname = data_dir / "reference/K03455.1.gtf"

accession_list_fname = data_dir / 'complete_sequences.2022.accessions'
summary_file = data_dir / "download.summary.tsv"

In [4]:
accessions = []
with open(accession_list_fname) as inFP:
    accessions = inFP.read().splitlines()
print(accessions[:5])

['K03455', 'MH705157', 'JQ403028', 'DQ396400', 'DQ676872']


In [39]:
fasta_url_template = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?db=nuccore&report=fasta&id={}"
gff3_url_template = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?db=nuccore&report=gff3&id={}"

def download_file(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            return True
        else:
            return False
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return False
    except Exception as err:
        print(f"An error occurred: {err}")
        return False
    
def load_min_max_features(gtf_fname):
    # load minimum and maximum for each coordinate type
    minmax = dict()
    with open(gtf_fname,"r") as inFP:
        for line in inFP:
            if line[0] == "#":
                continue
            lcs = line.strip().split("\t")
            minmax.setdefault(lcs[2],[int(lcs[3]),int(lcs[4])])
            minmax[lcs[2]][0] = min(minmax[lcs[2]][0],int(lcs[3]))
            minmax[lcs[2]][1] = max(minmax[lcs[2]][1],int(lcs[4]))

    # add min donor and max acceptor
    introns = get_intervals(gtf_fname,"exon",invert=True)
    for k, data in introns.items():
        for intron, tids in data.items():
            minmax.setdefault("donor",[intron[0],intron[0]])
            minmax.setdefault("acceptor",[intron[1],intron[1]])
            minmax["donor"][0] = min(minmax["donor"][0],intron[0])
            minmax["donor"][1] = max(minmax["donor"][1],intron[0])
            minmax["acceptor"][0] = min(minmax["acceptor"][0],intron[1])
            minmax["acceptor"][1] = max(minmax["acceptor"][1],intron[1])

    return minmax

    
def is_valid_genome(qry_fasta_fname, ref_fasta_fname, ref_gtf_fname, outdir) -> bool:
    # maps query to reference.
    # finds 3' and 5' clipping
    # checks if clipping includes any annotated regions
    # return False if genome is incomplete

    # map to alignments and add missing 5' and 3' information

    # load min/max intron and cds
    minmax = load_min_max_features(ref_gtf_fname)

    # align two genomes
    cmd = ["minimap2","-ax","map-ont",ref_fasta_fname,qry_fasta_fname]
    q2t_sam_fname = outdir / "q2t.sam"
    with open(q2t_sam_fname,"w+") as outFP:
        subprocess.call(cmd,stdout=outFP,stderr=subprocess.DEVNULL)

    q2t_bam_fname = outdir / "q2t.bam"
    cmd = ["samtools","view","-S","-b","-o",q2t_bam_fname,q2t_sam_fname]
    subprocess.call(cmd,stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

    cmd = ["samtools","index",q2t_bam_fname]
    subprocess.call(cmd,stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)

    # find 3' and 5' clippings
    bamfile = pysam.AlignmentFile(q2t_bam_fname, "rb")

    for read in bamfile:
        if not read.is_unmapped and not read.is_secondary and not (read.flag & 2048):
            ref_start = read.reference_start
            ref_end = read.reference_end

            break # should be just one alignment - exit after first. if mor ethan one - than likely fragmented and should not pass verification either
    
    if ref_start > minmax["donor"][0] or \
       ref_end < minmax["acceptor"][1] or \
       ref_start > minmax["CDS"][0] or \
       ref_end < minmax["CDS"][1]:
        return False, ref_start, ref_end
    
    return True, ref_start, ref_end

summary_data = []

for acc in accessions:
    fasta_url = fasta_url_template.format(acc)
    gff3_url = gff3_url_template.format(acc)

    fasta_file = outdir / f"{acc}.fasta"
    gff3_file = outdir / f"{acc}.gff3"

    fasta_downloaded = download_file(fasta_url, fasta_file)
    tmpdir = outdir / f"{acc}.tmp"
    tmpdir.mkdir(exist_ok=True)
    is_valid, ref_start, ref_end = is_valid_genome(fasta_file,reference_fasta_fname,reference_gtf_fname,tmpdir)
    if not is_valid:
        print(f"Genome {acc} is invalid because of clipping (start: {ref_start}, end: {ref_end})")
    shutil.rmtree(tmpdir) # remove tmpdir
    if not is_valid:
        # remove fasta file
        os.remove(fasta_file)
        continue

    gff3_downloaded = download_file(gff3_url, gff3_file)

    summary_data.append([acc, fasta_downloaded, gff3_downloaded])

with open(summary_file, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(["accession_id", "fasta", "gff"])
    writer.writerows(summary_data)

Genome JQ403028 is invalid because of clipping (start: 789, end: 9534)
Genome DQ676872 is invalid because of clipping (start: 789, end: 9414)
Genome MH746258 is invalid because of clipping (start: 788, end: 9631)
Genome MH746253 is invalid because of clipping (start: 788, end: 9604)
Genome FJ388893 is invalid because of clipping (start: 789, end: 8862)
Genome FJ388903 is invalid because of clipping (start: 789, end: 8922)
Genome FJ388909 is invalid because of clipping (start: 789, end: 8930)
Genome FJ388925 is invalid because of clipping (start: 789, end: 8930)
Genome FJ388938 is invalid because of clipping (start: 789, end: 8930)
Genome FJ388942 is invalid because of clipping (start: 789, end: 8930)
Genome FJ388943 is invalid because of clipping (start: 789, end: 8859)
Genome JF683759 is invalid because of clipping (start: 789, end: 8930)
Genome JF683760 is invalid because of clipping (start: 789, end: 8930)
Genome JF683767 is invalid because of clipping (start: 789, end: 8930)
Genome

In [5]:
def cleanup_hiv_gtf(input_gtf, output_gtf):
    # get cds chains for each transcript
    seqid = ""
    transcripts = {} # collect all unique CDS chains for each transcript
    tid2gid = {} # map transcript id to gene id
    with open(input_gtf, 'r') as inFP:
        for line in inFP:
            if line.startswith("#"):
                continue
        
            lcs = line.split("\t")

            attrs = extract_attributes(lcs[8])
            
            if lcs[2] == "transcript":
                gid = attrs["gene_id"]
                tid = attrs["transcript_id"]
                tid2gid[tid] = gid
                
            if lcs[2] != "CDS":
                continue

            seqid = lcs[0]
            tid = attrs["transcript_id"]
            if tid not in transcripts:
                transcripts[tid] = set()
            transcripts[tid].add(tuple([lcs[3],lcs[4],lcs[7]])) # start, end, phase

    # collect all unique CDS chains for each gene
    genes = {} # collect all unique CDS chains for each gene
    for tid, cds_chain in transcripts.items():
        gid = tid2gid[tid]
        if gid not in genes:
            genes[gid] = cds_chain
        assert genes[gid] == cds_chain, f"Gene {gid} has different CDS chains for different transcripts"

    # write out standardized GTF file
    with open(output_gtf+".tmp", 'w') as outFP:
        for gid, cds_chain in genes.items():
            for i, (start, end, phase) in enumerate(cds_chain):
                lcs = [seqid, "cleanup", "CDS", start, end, ".", "+", phase, "."]
                attrs = {"gene_id":gid,"transcript_id":gid}
                lcs[8] = to_attribute_string(attrs)
                outFP.write("\t".join(lcs)+"\n")

    # run gffread to generate the final gtf file
    cmd = ["gffread", "-E", "-T", "--force-exons", "-o", output_gtf, output_gtf+".tmp"]
    subprocess.run(cmd)

In [6]:
# iterate over the gff files and standardize them:
# - run gffread
# - parse atributes and assign appropriate gene_id and transcript_id

with open(summary_file,"r") as inFP:
    reader = csv.reader(inFP, delimiter="\t")
    # skip header
    next(reader)
    for (accid, fasta, gff) in reader:
        if accid == "K03455": # skip HXB2
            continue
        print(accid)
        
        fasta = bool(fasta)
        gff = bool(gff)
        if not fasta or not gff:
            print(f"Skipping {accid}")
            continue
        
        gff3_file = os.path.join(outdir, f"{accid}.gff3")
        gffread_fname = os.path.join(outdir, f"{accid}.gffread.gtf")
        cmd = ["gffread", "-E", "-T", "-o", gffread_fname, gff3_file]
        subprocess.call(cmd)
        
        gtf_fname = os.path.join(outdir, f"{accid}.gtf")
        
        cleanup_hiv_gtf(gffread_fname, gtf_fname)

MH705157


Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gff3
   .. loaded 10 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gtf /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gtf.tmp
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/MH705157.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/DQ396400.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ396400.gff3
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ396400.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/DQ396400.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ396400.gtf.tmp
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ396400.gtf.tmp
Command lin

DQ396400
KU168256
MH705158
MH705151
MH705153
MH705133
KU168305
MN153491
KP718918
KP718928
MT349418
FJ670519
KY496622
KT152846
KT152839
KT152840


   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KT152846.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/KT152846.gtf /home/avaraby1/HIV_Atlas/data/sequences/KT152846.gtf.tmp
   .. loaded 8 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KT152846.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/KT152839.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/KT152839.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KT152839.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/KT152839.gtf /home/avaraby1/HIV_Atlas/data/sequences/KT152839.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KT152839.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/KT152840.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/KT152840.gff3
   .. loaded

KT152844
EU110095
EU110088
EU110092
EU110094
KF716474
KF716475
AF539405
AF004885
AY322185
AY322190


...

Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/AB286849.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/AB286849.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AB286849.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/AB286849.gtf /home/avaraby1/HIV_Atlas/data/sequences/AB286849.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/AB286849.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/KU749418.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/KU749418.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KU749418.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/KU749418.gtf /home/avaraby1/HIV_Atlas/data/sequences/KU749418.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/KU749418.gtf.tmp
Command line

KY392770
GQ290462
KY359384
HM215251
HM215249
KY392780
KY392773
KY392775
AB485665
MN153492
KP718920
MN153489
JX140679
MK095236
AY046058
MH078559
EF029066
MN736701
MH705148


Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/MN153489.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/MN153489.gff3
   .. loaded 7 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/MN153489.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/MN153489.gtf /home/avaraby1/HIV_Atlas/data/sequences/MN153489.gtf.tmp
   .. loaded 7 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/MN153489.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/JX140679.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/JX140679.gff3
   .. loaded 5 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/JX140679.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/JX140679.gtf /home/avaraby1/HIV_Atlas/data/sequences/JX140679.gtf.tmp
   .. loaded 5 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/JX140679.gtf.tmp
Command line

KU168281
AY169812
KY112585
KU168288
KU168296
AY532635
DQ017382
DQ017383
MF767262
KY498771
AJ006022
AJ271370
JN572926
KY953207
AY169968
EF535993
DQ373065
DQ373064
DQ373063
EF535994
DQ373066
FR686511
AF115393
AJ271369
X52154
AF103818
FJ424871


   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ373065.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/DQ373065.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ373065.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ373065.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/DQ373064.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ373064.gff3
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ373064.gff3
Command line was:
gffread -E -T --force-exons -o /home/avaraby1/HIV_Atlas/data/sequences/DQ373064.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ373064.gtf.tmp
   .. loaded 9 genomic features from /home/avaraby1/HIV_Atlas/data/sequences/DQ373064.gtf.tmp
Command line was:
gffread -E -T -o /home/avaraby1/HIV_Atlas/data/sequences/DQ373063.gffread.gtf /home/avaraby1/HIV_Atlas/data/sequences/DQ373063.gff3
   .. loaded